<a href="https://colab.research.google.com/github/crislmfroes/pytorch-colab-notebooks/blob/master/PyTorch_Tutorial_DataLoadingAndProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip install -U torch pandas scikit-image matplotlib torchvision pillow==4.1.1 image

Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.0.0)
Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (0.23.4)
Requirement already up-to-date: scikit-image in /usr/local/lib/python3.6/dist-packages (0.14.1)
Requirement already up-to-date: matplotlib in /usr/local/lib/python3.6/dist-packages (3.0.2)
Requirement already up-to-date: torchvision in /usr/local/lib/python3.6/dist-packages (0.2.1)
    100% |████████████████████████████████| 5.7MB 4.6MB/s 
Requirement already up-to-date: image in /usr/local/lib/python3.6/dist-packages (1.5.27)
scikit-image 0.14.1 has requirement pillow>=4.3.0, but you'll have pillow 4.1.1 which is incompatible.
  Found existing installation: Pillow 4.3.0
    Uninstalling Pillow-4.3.0:
      Successfully uninstalled Pillow-4.3.0


In [0]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
plt.ion()


In [12]:
!wget https://download.pytorch.org/tutorial/faces.zip

--2018-12-11 00:25:08--  https://download.pytorch.org/tutorial/faces.zip
Resolving download.pytorch.org (download.pytorch.org)... 13.35.125.120, 13.35.125.93, 13.35.125.80, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.35.125.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5780252 (5.5M) [application/zip]
Saving to: ‘faces.zip’

faces.zip           100%[===================>]   5.51M  7.60MB/s    in 0.7s    

2018-12-11 00:25:09 (7.60 MB/s) - ‘faces.zip’ saved [5780252/5780252]



In [20]:
!ls data/faces

0805personali01.jpg	   3298715079_5af7c78fcb.jpg
1084239450_e76e00b7e7.jpg  3325611505_ddc7beffa1.jpg
10comm-decarlo.jpg	   3362762930_24f76cb89c.jpg
110276240_bec305da91.jpg   343583208_e986824d77.jpg
1198_0_861.jpg		   3461016494_56cce9c984.jpg
137341995_e7c48e9a75.jpg   348272697_832ce65324.jpg
1383023626_8a49e4879a.jpg  3534188114_2108895291.jpg
144044282_87cf3ff76e.jpg   3534189272_8ef88ba368.jpg
152601997_ec6429a43c.jpg   3555944509_7b477069c6.jpg
1549040388_b99e9fa295.jpg  3574737496_6ee8207045.jpg
1878519279_f905d4f34e.jpg  362167809_d5a5dcbfdb.jpg
2046713398_91aaa6fe1c.jpg  363149951_8be04dc6c0.jpg
2173711035_dbd53b4f9f.jpg  3638950581_3387685d3a.jpg
2210514040_6b03ff2629.jpg  3646828311_bfeb429ef7.jpg
2322901504_08122b01ba.jpg  3689162471_5f9ffb5aa0.jpg
2327253037_66a61ea6fe.jpg  3718903026_c1bf5dfcf8.jpg
2328398005_d328a70b4c.jpg  3790616528_297c0ac935.jpg
2370961440_6bc8ce346c.jpg  3855944735_e252959937.jpg
2382SJ8.jpg		   3856149136_d4595ffdd4.jpg
252418361_440b75751b.jpg 

In [21]:
landmarks_frame = pd.read_csv('data/faces/face_landmarks.csv')
n = 65
img_name = landmarks_frame.iloc[n, 0]
landmarks = landmarks_frame.iloc[n, 1:].as_matrix()
landmarks = landmarks.astype('float').reshape(-1,2)

print('Image name: {}'.format(img_name))
print('Landmarks shape: {}'.format(landmarks.shape))
print('First 4 landmarks: {}'.format(landmarks[:4]))

Image name: person-7.jpg
Landmarks shape: (68, 2)
First 4 landmarks: [[32. 65.]
 [33. 76.]
 [34. 86.]
 [34. 97.]]


In [0]:
class FaceLandmarksDataset(Dataset):
  def __init__(self, csv_file, root_dir, transform=None):
    self.landmarks_frame = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform
  def __len__(self):
    return len(self.landmarks_frame)
  def __getitem__(self, idx):
    img_name = os.path.join(self.root_dir, self.landmarks_frame.iloc[idx, 0])
    img = io.imread(img_name)
    landmarks = self.landmarks_frame.iloc[idx, 1:].as_matrix().astype('float').reshape(-1,2)
    sample = {'image': img, 'landmarks': landmarks}
    if self.transform:
      sample = self.transform(sample)
    return sample

In [0]:
!mkdir data
!mv faces data/faces

In [14]:
!unzip faces.zip

Archive:  faces.zip
   creating: faces/
  inflating: faces/0805personali01.jpg  
  inflating: faces/1084239450_e76e00b7e7.jpg  
  inflating: faces/10comm-decarlo.jpg  
  inflating: faces/110276240_bec305da91.jpg  
  inflating: faces/1198_0_861.jpg    
  inflating: faces/137341995_e7c48e9a75.jpg  
  inflating: faces/1383023626_8a49e4879a.jpg  
  inflating: faces/144044282_87cf3ff76e.jpg  
  inflating: faces/152601997_ec6429a43c.jpg  
  inflating: faces/1549040388_b99e9fa295.jpg  
  inflating: faces/1878519279_f905d4f34e.jpg  
  inflating: faces/2046713398_91aaa6fe1c.jpg  
  inflating: faces/2173711035_dbd53b4f9f.jpg  
  inflating: faces/2210514040_6b03ff2629.jpg  
  inflating: faces/2322901504_08122b01ba.jpg  
  inflating: faces/2327253037_66a61ea6fe.jpg  
  inflating: faces/2328398005_d328a70b4c.jpg  
  inflating: faces/2370961440_6bc8ce346c.jpg  
  inflating: faces/2382SJ8.jpg       
  inflating: faces/252418361_440b75751b.jpg  
  inflating: faces/262007783_943bbcf613.jpg  
  inflatin

In [38]:
face_dataset = FaceLandmarksDataset(csv_file='data/faces/face_landmarks.csv', root_dir='data/faces/')
for i in range(len(face_dataset)):
  sample = face_dataset[i]
  print(i, sample['image'].shape, sample['landmarks'].shape)

0 (324, 215, 3) (68, 2)
1 (500, 333, 3) (68, 2)
2 (250, 258, 3) (68, 2)
3 (434, 290, 3) (68, 2)
4 (828, 630, 3) (68, 2)
5 (402, 500, 3) (68, 2)
6 (332, 500, 3) (68, 2)
7 (333, 500, 3) (68, 2)
8 (334, 500, 3) (68, 2)
9 (376, 500, 3) (68, 2)
10 (333, 500, 3) (68, 2)
11 (500, 353, 3) (68, 2)
12 (500, 335, 3) (68, 2)
13 (500, 375, 3) (68, 2)
14 (375, 500, 3) (68, 2)
15 (500, 375, 3) (68, 2)
16 (500, 334, 3) (68, 2)
17 (500, 333, 3) (68, 2)
18 (303, 200, 3) (68, 2)
19 (333, 500, 3) (68, 2)
20 (500, 375, 3) (68, 2)
21 (333, 500, 3) (68, 2)
22 (500, 365, 3) (68, 2)
23 (375, 500, 3) (68, 2)
24 (333, 500, 3) (68, 2)
25 (333, 500, 3) (68, 2)
26 (500, 333, 3) (68, 2)
27 (500, 342, 3) (68, 2)
28 (500, 333, 3) (68, 2)
29 (500, 375, 3) (68, 2)
30 (500, 334, 3) (68, 2)
31 (332, 500, 3) (68, 2)
32 (333, 500, 3) (68, 2)
33 (500, 333, 3) (68, 2)
34 (332, 500, 3) (68, 2)
35 (319, 480, 3) (68, 2)
36 (333, 500, 3) (68, 2)
37 (500, 335, 3) (68, 2)
38 (333, 500, 3) (68, 2)
39 (500, 375, 3) (68, 2)
40 (334, 5

In [0]:
class Rescale(object):
  def __init__(self, output_size):
    assert isinstance(output_size, (int, tuple))
    self.output_size = output_size
  def __call__(self, sample):
    image, landmarks = sample['image'], sample['landmarks']
    h, w = image.shape[:2]
    if isinstance(self.output_size, int):
      if h > w:
        new_h, new_w = self.output_size * h / w, self.output_size
      else:
        new_h, new_w = self.output_size, self.output_size * w / h
    else:
      new_h, new_w = self.output_size
    new_h, new_w = int(new_h), int(new_w)
    img = transform.resize(image, (new_h, new_w))
    landmarks = landmarks * [new_w/w, new_h/h]
    return {'image': img, 'landmarks': landmarks}

In [0]:
class RandomCrop(object):
  def __init__(self, output_size):
    assert isinstance(output_size, (int, tuple))
    if isinstance(output_size, int):
      self.output_size = (output_size, output_size)
    else:
      assert len(output_size) == 2
      self.output_size = output_size
  def __call__(self, sample):
    image, landmarks = sample['image'], sample['landmarks']
    h, w = image.shape[:2]
    new_h, new_w = self.output_size
    top = np.random.randint(0, h - new_h)
    left = np.random.randint(0, w - new_h)
    image = image[top: top + new_h, left: left + new_w]
    landmarks = landmarks - [left, top]
    return {'image': image, 'landmarks': landmarks}


In [0]:
class ToTensor(object):
  def __call__(self, sample):
    image, landmarks = sample['image'], sample['landmarks']
    image = image.transpose((2, 0, 1))
    return {'image': torch.from_numpy(image), 'landmarks': torch.from_numpy(landmarks)}

In [43]:
scale = Rescale(256)
crop = RandomCrop(128)
composed = transforms.Compose([Rescale(256), RandomCrop(224)])
sample = face_dataset[65]
for i, trsfrm in enumerate([scale, crop, composed]):
  transformed_sample = trsfrm(sample)
  print(i, transformed_sample['image'].shape, transformed_sample['landmarks'].shape)

0 (256, 256, 3) (68, 2)
1 (128, 128, 3) (68, 2)
2 (224, 224, 3) (68, 2)


In [60]:
transformed_dataset = FaceLandmarksDataset(csv_file='data/faces/face_landmarks.csv', root_dir='data/faces',
                                          transform=transforms.Compose([
                                              Rescale(256),
                                              RandomCrop(224),
                                              ToTensor()
                                          ]))
for i in range(len(transformed_dataset)):
  sample = transformed_dataset[i]
  print(i, sample['image'].size(), sample['landmarks'].size())


0 torch.Size([3, 224, 224]) torch.Size([68, 2])
1 torch.Size([3, 224, 224]) torch.Size([68, 2])
2 torch.Size([3, 224, 224]) torch.Size([68, 2])
3 torch.Size([3, 224, 224]) torch.Size([68, 2])
4 torch.Size([3, 224, 224]) torch.Size([68, 2])
5 torch.Size([3, 224, 224]) torch.Size([68, 2])
6 torch.Size([3, 224, 224]) torch.Size([68, 2])
7 torch.Size([3, 224, 224]) torch.Size([68, 2])
8 torch.Size([3, 224, 224]) torch.Size([68, 2])
9 torch.Size([3, 224, 224]) torch.Size([68, 2])
10 torch.Size([3, 224, 224]) torch.Size([68, 2])
11 torch.Size([3, 224, 224]) torch.Size([68, 2])
12 torch.Size([3, 224, 224]) torch.Size([68, 2])
13 torch.Size([3, 224, 224]) torch.Size([68, 2])
14 torch.Size([3, 224, 224]) torch.Size([68, 2])
15 torch.Size([3, 224, 224]) torch.Size([68, 2])
16 torch.Size([3, 224, 224]) torch.Size([68, 2])
17 torch.Size([3, 224, 224]) torch.Size([68, 2])
18 torch.Size([3, 224, 224]) torch.Size([68, 2])
19 torch.Size([3, 224, 224]) torch.Size([68, 2])
20 torch.Size([3, 224, 224]) t

In [62]:
dataloader = DataLoader(transformed_dataset, batch_size=4, shuffle=True, num_workers=4)
for i_batch, sample_batched in enumerate(dataloader):
  print(i_batch, sample_batched['image'].size(), sample_batched['landmarks'].size())

0 torch.Size([4, 3, 224, 224]) torch.Size([4, 68, 2])
1 torch.Size([4, 3, 224, 224]) torch.Size([4, 68, 2])
2 torch.Size([4, 3, 224, 224]) torch.Size([4, 68, 2])
3 torch.Size([4, 3, 224, 224]) torch.Size([4, 68, 2])
4 torch.Size([4, 3, 224, 224]) torch.Size([4, 68, 2])
5 torch.Size([4, 3, 224, 224]) torch.Size([4, 68, 2])
6 torch.Size([4, 3, 224, 224]) torch.Size([4, 68, 2])
7 torch.Size([4, 3, 224, 224]) torch.Size([4, 68, 2])
8 torch.Size([4, 3, 224, 224]) torch.Size([4, 68, 2])
9 torch.Size([4, 3, 224, 224]) torch.Size([4, 68, 2])
10 torch.Size([4, 3, 224, 224]) torch.Size([4, 68, 2])
11 torch.Size([4, 3, 224, 224]) torch.Size([4, 68, 2])
12 torch.Size([4, 3, 224, 224]) torch.Size([4, 68, 2])
13 torch.Size([4, 3, 224, 224]) torch.Size([4, 68, 2])
14 torch.Size([4, 3, 224, 224]) torch.Size([4, 68, 2])
15 torch.Size([4, 3, 224, 224]) torch.Size([4, 68, 2])
16 torch.Size([4, 3, 224, 224]) torch.Size([4, 68, 2])
17 torch.Size([1, 3, 224, 224]) torch.Size([1, 68, 2])
